In [ ]:
# kg_logic.py
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import data_load 

def generate_scatter_matrix_plot(limit=5):
    # 1. 데이터 가져오기
    full_df = data_load.GLOBAL_DATA_CACHE.get("full_df")
    if full_df is None or full_df.empty: return None

    # 2. 전처리 (Top N개 센서 선정)
    sensor_map = {}
    for s, sub in full_df.groupby('sensor'):
        hist = []
        for h in sub['history']: hist.extend(h)
        sensor_map[s] = hist
        
    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in sensor_map.items() ])).dropna()

    if len(df.columns) > limit:
        top_cols = df.var().sort_values(ascending=False).head(limit).index
        df = df[top_cols]

    # 3. [핵심] Pandas로 상관계수 순식간에 계산
    corr_matrix = df.corr()
    
    # 4. [핵심] Plotly Python으로 복잡한 그리드 생성
    cols = df.columns
    n = len(cols)
    
    fig = make_subplots(rows=n, cols=n, print_grid=False, horizontal_spacing=0.02, vertical_spacing=0.02)

    for i, row_col in enumerate(cols):
        for j, col_col in enumerate(cols):
            r, c = i + 1, j + 1
            
            if i == j: # 대각선: 히스토그램
                fig.add_trace(go.Histogram(x=df[row_col], marker_color='#555'), row=r, col=c)
            elif i < j: # 상단: 스캐터
                fig.add_trace(go.Scattergl(x=df[col_col], y=df[row_col], mode='markers', marker=dict(size=3, color='#1f77b4', opacity=0.4)), row=r, col=c)
            else: # 하단: 상관계수 텍스트
                cv = corr_matrix.iloc[i, j]
                alpha = abs(cv)
                color = f'rgba(255, 0, 0, {alpha})' if cv > 0 else f'rgba(0, 0, 255, {alpha})'
                fig.add_trace(go.Scatter(x=[0.5], y=[0.5], mode='markers+text', marker=dict(size=100, symbol='square', color=color), text=[f"{cv:.2f}"], textfont=dict(size=16, color='white' if alpha > 0.5 else 'black')), row=r, col=c)
            
            fig.update_xaxes(showticklabels=False, row=r, col=c)
            fig.update_yaxes(showticklabels=False, row=r, col=c)
            if i == n - 1: fig.update_xaxes(title_text=col_col, title_font=dict(size=10), showticklabels=False, row=r, col=c)
            if j == 0: fig.update_yaxes(title_text=row_col, title_font=dict(size=10), showticklabels=False, row=r, col=c)

    fig.update_layout(margin=dict(l=40, r=20, t=20, b=40), showlegend=False, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='white')
    
    return json.loads(fig.to_json())

In [ ]:
# data_load.py

# ... (기존 코드) ...

# [신규] 상관분석용 데이터프레임 반환 (Top N개 센서만)
def get_sensor_dataframe_from_memory(limit=6):
    full_df = GLOBAL_DATA_CACHE.get("full_df")
    if full_df is None or full_df.empty: return None

    # 1. 센서별 시계열 통합
    sensor_history_map = {}
    for sensor_name, sub_df in full_df.groupby('sensor'):
        full_history = []
        for h in sub_df['history']: full_history.extend(h)
        sensor_history_map[sensor_name] = full_history

    if not sensor_history_map: return None

    # 2. DataFrame 생성
    ts_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in sensor_history_map.items() ]))
    ts_df = ts_df.dropna()

    # 3. [최적화] 센서가 너무 많으면 상관관계가 높은(변동성이 큰) 순서로 자르거나
    # 여기서는 편의상 앞에서부터 limit개만 자름
    if len(ts_df.columns) > limit:
        # 분산(Variance)이 큰 상위 N개 선택 (데이터 변화가 있는 센서 위주)
        top_cols = ts_df.var().sort_values(ascending=False).head(limit).index
        ts_df = ts_df[top_cols]

    # JSON 변환 (Plotly에서 쓸 수 있게 dict 형태)
    # { 'Top_Temp': [1,2,3...], 'Gas_Flow': [4,5,6...] }
    return ts_df.to_dict(orient='list')

In [ ]:
# main.py

# ...

@app.post("/api/correlation_scatter_matrix")
async def get_corr_scatter(req: KGRequest):
    import kg_logic
    # 상위 5개 센서만 사용하여 매트릭스 생성 (너무 많으면 브라우저 느려짐)
    chart_json = kg_logic.generate_scatter_matrix_plot(limit=5)
    
    if not chart_json:
        return JSONResponse({"error": "No data available"}, status_code=404)
        
    return JSONResponse(chart_json)

In [ ]:
/* static/css/style.css */

/* 기본 설정 유지 */
body { font-family: 'Segoe UI', sans-serif; background-color: #f4f6f9; margin: 0; padding: 0; display: flex; height: 100vh; overflow: hidden; }
.sidebar { width: 270px; min-width: 270px; background: #fff; border-right: 1px solid #ddd; padding: 20px; display: flex; flex-direction: column; gap: 12px; overflow-y: auto; z-index: 10; }

/* [레이아웃 변경] */
.content {
    flex: 1;
    display: flex;
    gap: 15px;
    padding: 15px;
    overflow: hidden;
}

/* 1. 좌측 패널 (Related Sensor Table) - 세로로 김 */
.left-panel {
    flex: 0 0 250px; /* 고정 너비 혹은 비율 */
    background: white;
    border-radius: 10px;
    border: 1px solid #e0e0e0;
    display: flex;
    flex-direction: column;
    overflow: hidden;
}

/* 2. 우측 패널 (상단/하단 분리) */
.right-panel {
    flex: 1;
    display: flex;
    flex-direction: column;
    gap: 15px;
    min-width: 0; /* Flexbox 버그 방지 */
}

/* 우측 상단 (Tree | Correlation) */
.right-top {
    flex: 1; /* 높이 비율 */
    display: flex;
    gap: 15px;
    min-height: 0;
}

#tree-container {
    flex: 1;
    background: white;
    border-radius: 10px;
    border: 1px solid #e0e0e0;
    position: relative;
}

#correlation-container {
    flex: 1;
    background: white;
    border-radius: 10px;
    border: 1px solid #e0e0e0;
    position: relative;
    display: flex;
    flex-direction: column;
}

/* 우측 하단 (Scatter/Line Plot) */
.right-bottom {
    flex: 1; /* 높이 비율 */
    background: white;
    border-radius: 10px;
    border: 1px solid #e0e0e0;
    position: relative;
    padding: 10px;
    display: flex;
    flex-direction: column;
}

/* 내부 요소 스타일 */
#list-header { padding: 12px; background: #fafafa; border-bottom: 1px solid #eee; font-weight: bold; font-size: 0.95rem; }
#list-content { flex: 1; overflow-y: auto; padding: 10px; }

/* 센서 아이템 스타일 */
.sensor-item {
    padding: 10px; margin-bottom: 6px; border-radius: 6px; cursor: pointer;
    border: 1px solid #eee; background: #fff;
    display: flex; justify-content: space-between; align-items: center;
    transition: all 0.2s;
}
.sensor-item:hover { background-color: #f5f5f5; transform: translateX(3px); }
.sensor-item.active { background-color: #e3f2fd; border-color: #2196f3; box-shadow: 0 2px 4px rgba(33,150,243,0.1); }

/* D3, Plotly, Labels 유지 */
.link-structure { stroke: #ccc; stroke-width: 1px; }
.link-correlation { stroke: #ff4444; stroke-width: 1px; stroke-dasharray: 4,2; }
.node-group { fill: #FFD700; stroke: #fff; stroke-width: 2px; }
.node-sensor { fill: #87CEEB; stroke: #fff; stroke-width: 1.5px; }
.edge-label { font-size: 9px; fill: #d32f2f; font-weight: bold; pointer-events: none; text-shadow: 1px 1px 0 #fff; }
.value-label-bg { fill: #2e7d32; opacity: 0.9; }
.value-label-text { font-size: 10px; fill: white; font-weight: bold; pointer-events: none; }

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>Advanced Sensor Dashboard</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
    <link rel="stylesheet" href="/static/css/style.css">
</head>
<body>

<!-- Sidebar (옵션) -->
<div class="sidebar">
    <h2>Analysis Options</h2>
    
    <!-- 1. Process -->
    <div class="control-group">
        <label>1. Process</label>
        <select id="process-select" onchange="updateToolList()">
            <option value="" disabled selected>Select Process</option>
            <option value="process1">Process 1</option>
            <option value="process2">Process 2</option>
            <option value="process3">Process 3</option>
        </select>
    </div>

    <!-- 2. Date -->
    <div class="control-group">
        <label>2. Date Range</label>
        <div style="display:flex; gap:5px;">
            <input type="date" id="start-date" value="2025-12-01" style="flex:1;">
            <input type="date" id="end-date" value="2025-12-10" style="flex:1;">
        </div>
    </div>

    <!-- 3. Tools -->
    <div class="control-group">
        <label>3. Tools (Multi-select)</label>
        <select id="tool-select" multiple onchange="updateTargetToolOptions()">
            <option disabled>Wait for process</option>
        </select>
    </div>

    <button onclick="updateRecipeList()" style="margin-bottom:10px; background:#f0f0f0; border:1px solid #ccc; cursor:pointer;">
        Get Recipes
    </button>

    <!-- 4. Recipes -->
    <div class="control-group">
        <label>4. Recipes (Multi-select)</label>
        <select id="recipe-select" multiple>
            <option disabled>Get Recipes first</option>
        </select>
    </div>

    <!-- Target Tool -->
    <div class="control-group" style="padding:10px; background:#ffebee; border:1px solid #ffcdd2; border-radius:4px;">
        <label style="color:#d32f2f;">★ Target Tool</label>
        <select id="target-tool-select">
            <option disabled selected>Select tools above</option>
        </select>
    </div>

    <button class="btn-primary" onclick="loadData()">Load Data & Visualize</button>

    <hr style="margin:15px 0; width:100%;">

    <!-- View Options -->
    <h3>View Controls</h3>
    <div class="control-group">
        <label>Issue Type</label>
        <select id="issue-select" onchange="refreshView()">
            <option value="Particle">Particle</option>
            <option value="Uniformity">Uniformity</option>
            <option value="Concentration">Concentration</option>
        </select>
    </div>
    <div class="control-group">
        <label>Correlation View</label>
        <div style="display:flex; gap:10px;">
            <label><input type="radio" name="corr-view" value="heatmap" checked onclick="switchCorrView('heatmap')"> Heatmap</label>
            <label><input type="radio" name="corr-view" value="graph" onclick="switchCorrView('graph')"> Graph</label>
        </div>
    </div>
</div>

<!-- Main Content -->
<div class="content">
    
    <!-- [1] 좌측: Related Sensor Table -->
    <div class="left-panel" id="list-wrapper">
        <div id="list-header">Related Sensors</div>
        <div id="list-content">
            <div style="color:#999; text-align:center; margin-top:50%;">
                Select a node from Tree to see related sensors here.
            </div>
        </div>
    </div>

    <!-- [2] 우측 패널 -->
    <div class="right-panel">
        
        <!-- [2-1] 우측 상단 -->
        <div class="right-top">
            <!-- Tree Graph -->
            <div id="tree-container">
                <div style="position:absolute; top:10px; left:10px; font-weight:bold; color:#555;">Hierarchy Tree</div>
                <div id="tree-wrapper" style="width:100%; height:100%;"></div>
            </div>
            
            <!-- Correlation Plot -->
            <div id="correlation-container">
                <div style="padding:10px; border-bottom:1px solid #eee; font-weight:bold; color:#555;">Correlation</div>
                <div id="corr-wrapper" style="flex:1; width:100%; height:100%;"></div>
            </div>
        </div>

        <!-- [2-2] 우측 하단: Scatter/Line Plot -->
        <div class="right-bottom">
            <div style="margin-bottom:5px; font-weight:bold; color:#555;">Detailed Trend / Scatter Plot</div>
            <div id="plotly-chart" style="width:100%; height:100%;"></div>
            <div id="placeholder" style="position:absolute; top:50%; left:50%; transform:translate(-50%, -50%); color:#999; font-size:1.2em;">
                Click a sensor from the Left List to view chart
            </div>
        </div>
    </div>
</div>

<script src="/static/js/script.js"></script>
</body>
</html>

In [ ]:
// static/js/script.js

// ... (상단 API 호출 함수들: updateToolList, updateRecipeList 등은 동일) ...

let currentCorrView = 'heatmap';

function switchCorrView(type) {
    currentCorrView = type;
    drawCorrelationAnalysis();
}

async function loadData() {
    // ... 파라미터 수집 ...
    const process = document.getElementById("process-select").value;
    const tools = Array.from(document.getElementById("tool-select").selectedOptions).map(o=>o.value);
    const recipes = Array.from(document.getElementById("recipe-select").selectedOptions).map(o=>o.value);
    const targetTool = document.getElementById("target-tool-select").value;

    if(!tools.length || !recipes.length) { alert("Select tools and recipes"); return; }
    
    const btn = document.querySelector(".btn-primary");
    btn.innerText = "Loading..."; btn.disabled = true;

    try {
        // 1. Load Data to Memory
        const res = await fetch("/api/load", {
            method: "POST", headers: {"Content-Type":"application/json"},
            body: JSON.stringify({
                process, tools, recipes,
                start_date: document.getElementById("start-date").value,
                end_date: document.getElementById("end-date").value,
                target_tool: targetTool
            })
        });
        if(!res.ok) throw new Error((await res.json()).error);
        
        const treeData = await res.json();
        
        // 2. Render Components
        renderTree(treeData, targetTool);
        drawCorrelationAnalysis(); // Memory 기반

    } catch(e) { alert(e.message); }
    finally { btn.innerText = "Load Data & Visualize"; btn.disabled = false; }
}

// [신규] 메모리 기반 상관분석
async function drawCorrelationAnalysis() {
    const wrapper = document.getElementById("corr-wrapper");
    wrapper.innerHTML = "<div style='text-align:center; padding-top:20px;'>Loading...</div>";

    try {
        // [Graph View] -> Scatter Matrix (Python Backend Rendering)
        if (currentCorrView === 'graph') {
            const res = await fetch("/api/correlation_scatter_matrix", { 
                method: "POST", 
                headers: {"Content-Type":"application/json"}, 
                body: JSON.stringify({
                    process: "", tools: [], recipes: [], start_date: "", end_date: "", threshold: 0 
                }) 
            });
            
            if(res.ok) {
                const fig = await res.json();
                wrapper.innerHTML = "";
                Plotly.newPlot('corr-wrapper', fig.data, fig.layout, {responsive: true});
            } else {
                wrapper.innerHTML = "<div style='text-align:center; color:#999; padding-top:20px;'>No data available</div>";
            }
        } 
        // [Heatmap View] -> Simple Heatmap (Python Backend Calculation)
        else if (currentCorrView === 'heatmap') {
            const res = await fetch("/api/correlation_matrix", {
                method: "POST", headers: {"Content-Type":"application/json"},
                body: JSON.stringify({ process:"", tools:[], recipes:[], start_date:"", end_date:"", threshold:0 })
            });

            if(res.ok) {
                const data = await res.json();
                wrapper.innerHTML = "";
                drawPlotlyHeatmap(data, 'corr-wrapper');
            } else {
                wrapper.innerHTML = "<div style='text-align:center; color:#999; padding-top:20px;'>No data available</div>";
            }
        }
    } catch(e) { 
        console.error(e); 
        wrapper.innerHTML = "<div style='text-align:center; color:red; padding-top:20px;'>Error loading chart</div>"; 
    }
}

function drawPlotlyHeatmap(data, divId) {
    const trace = {
        z: data.z, x: data.x, y: data.y,
        type: 'heatmap', colorscale: 'RdBu', zmin: -1, zmax: 1
    };
    const layout = {
        margin: {t:10, r:10, b:40, l:40},
        paper_bgcolor: 'rgba(0,0,0,0)',
        xaxis: { ticks: '', side: 'bottom' },
        yaxis: { ticks: '', autorange: 'reversed' }
    };
    Plotly.newPlot(divId, [trace], layout, {responsive: true, displayModeBar: false});
}


function renderTree(treeData, targetTool) {
    d3.select("#tree-wrapper").selectAll("*").remove();
    
    // 리스트 초기화
    document.getElementById("list-content").innerHTML = "<div style='color:#999; text-align:center; margin-top:20px;'>Select a node from Tree</div>";
    
    const container = document.getElementById("tree-wrapper");
    const width = container.clientWidth;
    const height = container.clientHeight;

    const svg = d3.select("#tree-wrapper").append("svg")
        .attr("width", "100%").attr("height", "100%")
        .call(d3.zoom().on("zoom", (e) => g.attr("transform", e.transform)))
        .on("dblclick.zoom", null);
    const g = svg.append("g");
    
    const root = d3.hierarchy(treeData, d => d.children);
    const tree = d3.tree().nodeSize([40, 120]);
    const nodes = tree(root);

    const link = g.selectAll(".link").data(nodes.links()).enter().append("path")
        .attr("class", "link").attr("d", d3.linkHorizontal().x(d => d.y).y(d => d.x));

    const node = g.selectAll(".node").data(nodes.descendants()).enter().append("g")
        .attr("class", "node").attr("transform", d => `translate(${d.y},${d.x})`);

    node.append("circle").attr("r", 8)
        .on("click", function(event, d) {
            d3.selectAll("circle").classed("selected", false);
            d3.select(this).classed("selected", true);
            const ancestors = new Set(d.ancestors());
            link.classed("highlight", l => ancestors.has(l.target)).classed("dimmed", l => !ancestors.has(l.target));

            // [좌측 패널] Related Sensor List 채우기
            const listContent = document.getElementById("list-content");
            const listHeader = document.getElementById("list-header");
            listContent.innerHTML = "";

            if (d.data.related_sensors && d.data.related_sensors.length > 0) {
                listHeader.innerHTML = `${d.data.name}<br><small style='color:#d32f2f;'>Sorted by Diff</small>`;
                
                d.data.related_sensors.forEach(sensor => {
                    const item = document.createElement("div");
                    item.className = "sensor-item";
                    const diffVal = sensor.diff_value ? Math.round(sensor.diff_value * 10)/10 : 0;
                    
                    item.innerHTML = `
                        <div style="display:flex; flex-direction:column;">
                            <span style="font-weight:600;">${sensor.name}</span>
                        </div>
                        <span style="font-size:0.85em; color:${diffVal > 10 ? 'red':'#666'}; font-weight:bold;">
                            Diff: ${diffVal}
                        </span>`;
                    
                    // 리스트 아이템 클릭 시 -> 우측 하단 차트 그리기
                    item.onclick = async () => {
                        document.querySelectorAll(".sensor-item").forEach(el => el.classList.remove("active"));
                        item.classList.add("active");
                        
                        await fetchAndDrawChart(sensor.name, targetTool);
                    };
                    listContent.appendChild(item);
                });
            } else {
                listHeader.innerText = "No Data";
                listContent.innerHTML = "<div style='padding:20px; text-align:center; color:#ccc;'>No related sensors available</div>";
            }

            // 노드 클릭 시 해당 센서의 차트도 바로 그림 (선택사항)
            if (d.data.type === 'sensor') {
                fetchAndDrawChart(d.data.name, targetTool);
            }

            event.stopPropagation();
        });

    node.append("text").attr("dy", ".35em").attr("x", d => d.children ? -12 : 12)
        .style("text-anchor", d => d.children ? "end" : "start").text(d => d.data.name);

    svg.call(d3.zoom().transform, d3.zoomIdentity.translate(40, height / 2));
}

// [공통] 차트 데이터 요청 및 그리기 함수 (우측 하단)
async function fetchAndDrawChart(sensorName, targetTool) {
    const placeholder = document.getElementById("placeholder");
    placeholder.style.display = "block";
    placeholder.innerText = "Loading Chart...";
    Plotly.purge('plotly-chart');

    try {
        const res = await fetch("/api/chart", {
            method: "POST", headers: {"Content-Type": "application/json"},
            body: JSON.stringify({ sensor_name: sensorName })
        });
        
        if(res.ok) {
            const chartData = await res.json();
            placeholder.style.display = "none";
            // 색상 등 스타일을 타겟 툴에 맞춰 조정하려면 백엔드에서 생성할 때 target_tool 정보를 쓰거나
            // 여기서 layout을 수정할 수 있음. (현재는 백엔드 plot_func에서 처리됨)
            Plotly.newPlot('plotly-chart', chartData.data, chartData.layout, {responsive: true});
        }
    } catch(e) { console.error(e); }
}

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px # 컬러 스케일 사용을 위해

# ---------------------------------------------------------
# 1. 샘플 데이터 생성
# ---------------------------------------------------------
sensors = ['Top_Temp', 'Bottom_Temp', 'Gas_Flow', 'Pressure', 'Power']
n_steps = 150
data = {}
base = np.linspace(0, 10, n_steps)

for s in sensors:
    noise = np.random.normal(0, 2, n_steps)
    if 'Temp' in s:
        data[s] = base + noise + 20
    elif 'Gas' in s:
        data[s] = (base * -1) + noise + 50 # 음의 상관관계 유도
    else:
        data[s] = np.random.normal(50, 10, n_steps) # 랜덤

df = pd.DataFrame(data)
cols = df.columns
n_vars = len(cols)

# 상관계수 계산
corr_matrix = df.corr()

# ---------------------------------------------------------
# 2. Subplots 설정 (N x N 그리드)
# ---------------------------------------------------------
fig = make_subplots(
    rows=n_vars, cols=n_vars,
    print_grid=False,
    horizontal_spacing=0.02,
    vertical_spacing=0.02
)

# ---------------------------------------------------------
# 3. 그래프 채우기 (Loop)
# ---------------------------------------------------------
for i, row_col in enumerate(cols):      # i: 행 (Row)
    for j, col_col in enumerate(cols):  # j: 열 (Col)
        
        # Plotly는 인덱스가 1부터 시작함
        r, c = i + 1, j + 1
        
        # 1) 대각선 (Diagonal): 히스토그램 (분포)
        if i == j:
            fig.add_trace(
                go.Histogram(
                    x=df[row_col], 
                    marker_color='#333',
                    showlegend=False
                ),
                row=r, col=c
            )
            
        # 2) 상단 (Upper Triangle): Scatter Plot
        elif i < j:
            fig.add_trace(
                go.Scatter(
                    x=df[col_col], 
                    y=df[row_col],
                    mode='markers',
                    marker=dict(size=4, color='#1f77b4', opacity=0.6),
                    showlegend=False
                ),
                row=r, col=c
            )
            
        # 3) 하단 (Lower Triangle): Correlation Value & Color
        else:
            corr_val = corr_matrix.iloc[i, j]
            
            # 색상 결정 (음수:파랑, 양수:빨강, 0:흰색)
            # 투명도(alpha)로 강도 표현
            alpha = abs(corr_val)
            if corr_val > 0:
                color = f'rgba(255, 0, 0, {alpha})' # Red
            else:
                color = f'rgba(0, 0, 255, {alpha})' # Blue
            
            # 배경색을 칠하는 대신, 아주 큰 네모 마커를 뒤에 깔아서 색상 표현
            fig.add_trace(
                go.Scatter(
                    x=[0.5], y=[0.5], # 중앙 정렬
                    mode='markers+text',
                    marker=dict(size=1000, color=color, symbol='square'), # 배경 역할
                    text=[f"{corr_val:.2f}"], # 텍스트 표시
                    textfont=dict(size=14, color='black' if alpha < 0.5 else 'white'),
                    hoverinfo='text',
                    showlegend=False
                ),
                row=r, col=c
            )
            
            # 하단 텍스트 그래프는 축 눈금 숨기기
            fig.update_xaxes(showgrid=False, showticklabels=False, zeroline=False, range=[0, 1], row=r, col=c)
            fig.update_yaxes(showgrid=False, showticklabels=False, zeroline=False, range=[0, 1], row=r, col=c)

# ---------------------------------------------------------
# 4. 레이아웃 다듬기
# ---------------------------------------------------------
# 축 라벨 추가 (맨 왼쪽과 맨 아래쪽만)
for i, col in enumerate(cols):
    # 맨 아래쪽 행 (x축 라벨)
    fig.update_xaxes(title_text=col, row=n_vars, col=i+1)
    # 맨 왼쪽 열 (y축 라벨)
    fig.update_yaxes(title_text=col, row=i+1, col=1)

fig.update_layout(
    title="Scatter Plot Matrix with Correlation Heatmap",
    width=800,
    height=800,
    showlegend=False,
    plot_bgcolor='white'
)

fig.show()